# Assignment 2 Part 3 (Linear Programming)

## Model
- Decision variables: $x_{ia}$ (expected number of times an action a is taken in state i)
- Objective function: $\sum_{i, j} x_{ij}*r_{ij} $
- We need to find $x_{ij}$ for all i, j (this will be our decision rule or policy) with an objective to maximize the above objective function 
- Constraints: 
    1. $x_{ij} >= 0.0 \; \; \;  \forall i, j $ (As probabilities have to be positive)
    2. $\sum_{a} x_{ja}  -  \sum_{i}\sum_{a}x_{ia}p_{ij}^{a} = \alpha_{j} \; \; \; \; \forall j$ 

In [1]:
import numpy as np
import cvxpy as cp
# X matrix (decision rules)
n_states = 5   # N, S, E, W, C
n_actions = 9  # L, R, U, D, stay, gather, craft, shoot, hit
X = np.zeros(n_states * n_actions)
R = np.zeros(n_states * n_actions)

In [14]:
class State:
    def __init__(self):
        self.meta_data = {
            "position": ['W', 'N', 'E', 'S', 'C'],
            "material": [0, 1, 2],
            "no_of_arrow": [0, 1],
            "MM_state": ['D', 'R'],
            "MM_health": [0, 25, 50, 75, 100],
            "action": ['UP', 'LEFT', 'DOWN', 'RIGHT', 'STAY', 'SHOOT', 'HIT', 'CRAFT', 'GATHER', 'NONE']
        }
        self.state = {}


# Meta data declaration

In [15]:
meta_data = {
            "position": ['W', 'N', 'E', 'S', 'C'],
            "material": [0, 1, 2],
            "no_of_arrow": [0, 1],
            "MM_state": ['D', 'R'],
            "MM_health": [0, 25, 50, 75, 100],
            "action": ['UP', 'LEFT', 'DOWN', 'RIGHT', 'STAY', 'SHOOT', 'HIT', 'CRAFT', 'GATHER', 'NONE']
        }

# Return valid list of states

In [16]:
def return_valid_lists():
    valid_states = []
    #k = 0
    for position in meta_data['position']:
        for material in meta_data['material']:
            for n_arrows in meta_data['no_of_arrow']:
                for mm_state in meta_data['MM_state']:
                    for mm_health in meta_data['MM_health']:
                        for action in meta_data['action']:
                            #k+=1
                            # North
                            if position=="N":
                                if mm_health == 0:
                                    continue
                                if action=="DOWN" or action=="STAY" or action=="NONE":
                                    valid_states.append([position, material, n_arrows, mm_state, mm_health, action])
                                    continue
                                if material>=1 and action=="CRAFT":
                                    valid_states.append([position, material, n_arrows, mm_state, mm_health, action])
                                    continue
                            # South   
                            if position=="S":
                                if mm_health == 0:
                                    continue
                                if action=="UP" or action=="STAY" or action=="NONE" or action=="GATHER" :
                                    valid_states.append([position, material, n_arrows, mm_state, mm_health, action])
                                    continue
                                #pass
                            #East
                            if position=="E":
                                if action=="LEFT" or action=="STAY" or action=="NONE" or action=="HIT":
                                    valid_states.append([position, material, n_arrows, mm_state, mm_health, action])
                                    continue
                                if action=="SHOOT" and n_arrows>0:
                                    valid_states.append([position, material, n_arrows, mm_state, mm_health, action])
                                    continue
                                #pass
                            # West
                            if position=="W":
                                if action=="RIGHT" or action=="STAY" or action=="NONE":
                                    valid_states.append([position, material, n_arrows, mm_state, mm_health, action])
                                    continue
                                if action=="SHOOT" and n_arrows>0:
                                    valid_states.append([position, material, n_arrows, mm_state, mm_health, action])
                                    continue
                                #pass
                            # Centre
                            if position=="C":
                                if action=="RIGHT" or action=="LEFT" or action=="UP" or action=="DOWN" or action=="STAY" or action=="NONE":
                                    valid_states.append([position, material, n_arrows, mm_state, mm_health, action])
                                    continue
                                if action=="SHOOT" and n_arrows>0:
                                    valid_states.append([position, material, n_arrows, mm_state, mm_health, action])
                                    continue               
                                #pass
    #print(k)
    return valid_states

# Initialize X and R vectors

In [17]:
def init_X_and_R():
    output_meta_data = []
    X = []
    R = []
    valid_states = return_valid_lists()
    #print(len(valid_states))
    for position in meta_data['position']:
        for material in meta_data['material']:
            for n_arrows in meta_data['no_of_arrow']:
                for mm_state in meta_data['MM_state']:
                    for mm_health in meta_data['MM_health']:
                        for action in meta_data['action']:
                            a = [position, material, n_arrows, mm_state, mm_health, action]
                            if a in valid_states:
                                output_meta_data.append([ position, material, n_arrows, mm_state, mm_health, action ])
                                X.append(0.0)
                                R.append(0.0)
    return X, R, output_meta_data

# Update R vector

In [18]:
def update_R(output_meta_data):
    R = []
    for record in output_meta_data:
        # record = [position, material, n_arrows, mm_state, mm_health, action]
        #print(record)
        #print(record[0])
        if record[0]=='N' or record[0]=='S':
            R.append(-1.0)
        if record[0]=='E' or record[0]=='W' or record[0]=='C':
            if record[4]==0:
                R.append(0.0)
            elif record[3]=='D':
                R.append(-1.0)
            elif record[3]=='R' and record[5]!='STAY':
                R.append(0.5*(-40) + 0.5*(-1))
            elif record[5]=='STAY':
                R.append(-1)
            else:
                print("WTF")
            #pass
    return R

# Create A matrix

In [ ]:
def init_A_matrix():
    

# Test helper functions here

In [19]:

    #pass
X, R, output_meta_data = init_X_and_R()
print("Output meta size: "+str(len(output_meta_data)))
print(output_meta_data)
R = update_R(output_meta_data)
print(len(R))
print(R)
print(len(X))
#print(len(output_meta_data))
#print(X)
#print(R)

Output meta size: 1238
[['W', 0, 0, 'D', 0, 'RIGHT'], ['W', 0, 0, 'D', 0, 'STAY'], ['W', 0, 0, 'D', 0, 'NONE'], ['W', 0, 0, 'D', 25, 'RIGHT'], ['W', 0, 0, 'D', 25, 'STAY'], ['W', 0, 0, 'D', 25, 'NONE'], ['W', 0, 0, 'D', 50, 'RIGHT'], ['W', 0, 0, 'D', 50, 'STAY'], ['W', 0, 0, 'D', 50, 'NONE'], ['W', 0, 0, 'D', 75, 'RIGHT'], ['W', 0, 0, 'D', 75, 'STAY'], ['W', 0, 0, 'D', 75, 'NONE'], ['W', 0, 0, 'D', 100, 'RIGHT'], ['W', 0, 0, 'D', 100, 'STAY'], ['W', 0, 0, 'D', 100, 'NONE'], ['W', 0, 0, 'R', 0, 'RIGHT'], ['W', 0, 0, 'R', 0, 'STAY'], ['W', 0, 0, 'R', 0, 'NONE'], ['W', 0, 0, 'R', 25, 'RIGHT'], ['W', 0, 0, 'R', 25, 'STAY'], ['W', 0, 0, 'R', 25, 'NONE'], ['W', 0, 0, 'R', 50, 'RIGHT'], ['W', 0, 0, 'R', 50, 'STAY'], ['W', 0, 0, 'R', 50, 'NONE'], ['W', 0, 0, 'R', 75, 'RIGHT'], ['W', 0, 0, 'R', 75, 'STAY'], ['W', 0, 0, 'R', 75, 'NONE'], ['W', 0, 0, 'R', 100, 'RIGHT'], ['W', 0, 0, 'R', 100, 'STAY'], ['W', 0, 0, 'R', 100, 'NONE'], ['W', 0, 1, 'D', 0, 'RIGHT'], ['W', 0, 1, 'D', 0, 'STAY'], ['W', 0